# genetable to array test using panda and numpy

In [15]:
import requests, json
import pandas as pd
import numpy as np

In [6]:
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
lists="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"

In [85]:
'''Use the genes from the summary file to search for Knetminer genome api.'''
def knetapi(disc_genes):
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(lists, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
            #print("The phenotype of interests are {}".format(pheno))
            for line in fs:
                col = line.split("\t")
                genes.append(col[0])
            genelist = (",").join(genes) #join all iterative elements by ,
            #print(genelist)

            #use str.join() to convert multiple elments in a list into one string.
            keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)

            #check if request is succesfsul.
            if not r.ok:
                r.raise_for_status()
                sys.exit()
             #decoded is the genetable in string form.
            decoded = str(r.json()[u'geneTable']).rstrip()
            genetablist = decoded.split("\t")
            #print(type(decoded))

            #decoded is now a list of all the strings in genetable separated by tab.
            genearray = np.array(genetablist)
            print(genearray)
            print("\n" * 2)
            print(len(genearray))
            
            print("Without decoding")
            print(type(r.json()[u'geneTable']))
    #end of knetapi

##### Fundamental of converting string to list

In [26]:
string="hello{}world".format("\t")
print(string)

hello	world


In [27]:
print(list(string))

['h', 'e', 'l', 'l', 'o', '\t', 'w', 'o', 'r', 'l', 'd']


In [29]:
conversion = string.split("\t")
print(conversion)
print(type(conversion))

['hello', 'world']
<type 'list'>


#### Back to the main problem

In [86]:
knetapi(disc_genes)

['ONDEX-ID' 'ACCESSION' 'GENE NAME' 'CHRO' 'START' 'TAXID' 'SCORE' 'USER'
 'QTL' 'EVIDENCE\n47205' 'OS03G0122600' 'MADS50' '3' '1270320' '39947'
 '105.49' 'yes'
 'qtaro_301//qtaro_301||qtaro_531//qtaro_531||qtaro_284//qtaro_284||qtaro_900//qtaro_900||qtaro_282//qtaro_282||qtaro_297//qtaro_297||qtaro_530//qtaro_530||qtaro_153//qtaro_153||qtaro_1041//qtaro_1041||qtaro_731//qtaro_731'
 'Phenotype__1__Flowering time independent of...||Gene__2__SOC1//AGL13||BioProc__1__response to long-day, floweri...||Publication__23__PMID:26839287//PMID:24019760//PMID:23636663//PMID:19921247//PMID:19137708//PMID:19747386//PMID:18849573//PMID:18258861//PMID:18571125//PMID:17660946//PMID:15144377//PMID:14605222//PMID:10995392//PMID:10834834//PMID:10382970//PMID:9721681//PMID:8980519//PMID:9065695//PMID:8787738//PMID:8900276||TO__2__leaf length//root length||Protein__1__Q9XJ60\n74421'
 'OS06G0156700' 'OS06G0156700' '6' '2898787' '39947' '3.69' 'yes'
 'qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_666//qtaro_

##### The genetable file has 54 elements instead of 55 here. So this means it's definitely risky to decode utf-8 unicode of genetable into string with str()

The last element is a newline

##### Best practice of decoding utf-8 unicode here is to use .decode() method.

In [82]:
string = "This is a string"
string = string.encode(encoding="utf-8",errors="strict")
print(string)

This is a string


# New approaches based on intel above

 - genetable consists of 9 columns.
 - The ones we actually extract:
     - Accession
     - Score

Pandas dataframe can take either dictionary or lists as input.

### Method 1


1) Input Genome.json, a dictionary object into pandas and make the dataframe


2) Identify the accession and score columns in dataframe genetable.


3) Extract and decode those 2 fields.
 
### Method 2


1) Obtain genetable unicode from genome.json dictionary


2) decode genetable into string


3) create a tab separated list of genetable


4) Create a dataframe from that


5) Extract accession and score

# Method 1

## Does not work because lists of dictionary not one dictionary needs to be the input.

In [90]:
import requests, json
import pandas as pd
import numpy as n
from collections import OrderedDict

In [141]:
dicts = {"key1":0, 
        "key2":1,
        "key3":2 }
pd.DataFrame(dicts, index=[0])

,key1,key2,key3
0,0,1,2


In [91]:
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
lists="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"

In [154]:
'''Use the genes from the summary file to search for Knetminer genome api.'''
def knetapi(disc_genes):
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(lists, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
            #print("The phenotype of interests are {}".format(pheno))
            for line in fs:
                col = line.split("\t")
                genes.append(col[0])
            genelist = (",").join(genes) #join all iterative elements by ,
            #print(genelist)

            #use str.join() to convert multiple elments in a list into one string.
            keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)

            #check if request is succesfsul.
            if not r.ok:
                r.raise_for_status()
                sys.exit()
             # decoded json is a dictionary
            decoded = r.json()
            #print(type(decoded))
            #print(decoded)
            
            #Dataframe with keys being the index
            df = pd.DataFrame(decoded, index=[0], column="geneTable")
            print(df)
            
            

In [155]:
knetapi(disc_genes)

TypeError: __init__() got an unexpected keyword argument 'column'

# Method 2

In [119]:
import requests, json
import pandas as pd
import numpy as np

In [120]:
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
lists="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"

In [130]:
'''Use the genes from the summary file to search for Knetminer genome api.'''
def knetapi(disc_genes):
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(lists, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
            #print("The phenotype of interests are {}".format(pheno))
            for line in fs:
                col = line.split("\t")
                genes.append(col[0])
            genelist = (",").join(genes) #join all iterative elements by ,
            #print(genelist)

            #use str.join() to convert multiple elments in a list into one string.
            keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)

            #check if request is succesfsul.
            if not r.ok:
                r.raise_for_status()
                sys.exit()
             #decoded is the genetable in string form.
            decoded = str(r.json()[u'geneTable']).rstrip()
            genetablist = decoded.split("\t")
            print(type(genetablist))
            
            geneframe = pd.DataFrame(genetablist)
            print(geneframe)
            
        

    
    
    
knetapi(disc_genes)


<type 'list'>
                                                    0
0                                            ONDEX-ID
1                                           ACCESSION
2                                           GENE NAME
3                                                CHRO
4                                               START
5                                               TAXID
6                                               SCORE
7                                                USER
8                                                 QTL
9                                     EVIDENCE\n47205
10                                       OS03G0122600
11                                             MADS50
12                                                  3
13                                            1270320
14                                              39947
15                                             105.49
16                                                yes
17  qtaro_301/

# Pandas learning

http://pandas.pydata.org/pandas-docs/version/0.15/10min.html

In [195]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

In [202]:
r = requests.get("http://knetminer.rothamsted.ac.uk/riceknet/genome?list=OS06G0156600,OS06G0156600,OS06G0166100&keyword=(Mesocotyl+AND+length)")
#print(type(r.json()))
decoded = r.json()
genetab = decoded[u'geneTable']
genetab2 = str(genetab).rstrip()
listform = genetab2.split("\t")


#tweaking and inspecting data
print(listform)
print(type(listform))
print("\n")
print(listform[10:])
print("\n")
print("\n")

#pd.DataFrame(listform[10:])

print("\n")
pd.concat([pd.read_table(StringIO(g), sep = "\t") for g in genetab])

['ONDEX-ID', 'ACCESSION', 'GENE NAME', 'CHRO', 'START', 'TAXID', 'SCORE', 'USER', 'QTL', 'EVIDENCE\n74416', 'OS06G0156600', 'OS06G0156600', '6', '2893694', '39947', '0.00', 'yes', 'qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_666//qtaro_666||qtaro_600//qtaro_600||qtaro_2//qtaro_2||qtaro_607//qtaro_607||qtaro_501//qtaro_501||qtaro_448//qtaro_448||qtaro_674//qtaro_674||qtaro_817//qtaro_817', '\n74630', 'OS06G0166100', 'OS06G0166100', '6', '3322909', '39947', '0.00', 'yes', 'qtaro_399//qtaro_399||qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_234//qtaro_234||qtaro_666//qtaro_666||qtaro_2//qtaro_2||qtaro_607//qtaro_607||qtaro_400//qtaro_400||qtaro_448//qtaro_448||qtaro_674//qtaro_674||qtaro_401//qtaro_401||qtaro_817//qtaro_817']
<type 'list'>


['OS06G0156600', 'OS06G0156600', '6', '2893694', '39947', '0.00', 'yes', 'qtaro_447//qtaro_447||qtaro_4//qtaro_4||qtaro_666//qtaro_666||qtaro_600//qtaro_600||qtaro_2//qtaro_2||qtaro_607//qtaro_607||qtaro_501//qtaro_501||qtaro_448//qtaro_448||qtaro_6

EmptyDataError: No columns to parse from file

### reading from string?

https://stackoverflow.com/questions/22604564/create-pandas-dataframe-from-a-string

https://stackoverflow.com/questions/42789933/python-converting-unicode-in-list-to-dataframe

In [208]:
pd.concat([pd.read_csv(StringIO(r), sep = ";") for r in r.text])

ParserError: Error tokenizing data. C error: EOF inside string starting at line 0

In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

In [211]:
r = requests.get("http://knetminer.rothamsted.ac.uk/riceknet/genome?list=OS06G0156600,OS06G0156600,OS06G0166100&keyword=(Mesocotyl+AND+length)")
#print(type(r.json()))
print(type(r.text))
data = r.text

pd.concat([pd.read_json(StringIO(d)) for d in data])

<type 'unicode'>


ValueError: Expected object or value